# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
!pip3 install pandas
!pip3 install plotly
!pip3 install numpy
!pip3 install seaborn
!pip3 install matplotlib.pyplot
!pip3 install mpl_toolkits
!pip3 install sklearn
!pip3 install statsmodels
!pip3 install random
!pip3 install time

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user install

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [15]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.compose import make_column_transformer

import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from random import shuffle, seed

In [16]:
clientdata = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [17]:
clientdata.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [18]:
# clean data
clientdata = clientdata.dropna()

# Consider values that can be made binary, from skimming the data we can make the following changes: 
# - default: the few "yes" values can be removed, and "no" / "unknown" values can be converted into binary values
# this column can now represent whether the customer has NO credit in default
# - housing: drop "unknown"
# - load: drop "unknown"

clientdata_copy = clientdata

# default field cleanup / new encoding
clientdata_copy = clientdata_copy[clientdata_copy['default'] != 'yes'] 
new_encodings_default = {'no' : 1, 'unknown' : 0}
clientdata_copy = clientdata_copy.rename(columns={'default' : 'no_default_credit'})
clientdata_copy['no_default_credit'] = clientdata_copy['no_default_credit'].map(new_encodings_default)

# housing field cleanup 
clientdata_copy = clientdata_copy.query("(housing != 'unknown')")

# loan field cleanup 
clientdata_copy = clientdata_copy.query("(loan != 'unknown')")

# reset indices
clientdata_copy = clientdata_copy.reset_index(drop=True)
clientdata_copy.head()

data = clientdata_copy # cleaned data
data.head()

,age,job,marital,education,no_default_credit,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,1,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,0,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,1,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,1,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,1,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [19]:
# Preliminary Analysis - lets see what the demographic of some of the successful campaigns are to better understand
# our data! 
only_success = data[data['y'] == 'yes']

successful_outcome_job_dist = px.histogram(only_success, x="job")
successful_outcome_job_dist.show()

successful_outcome_marital_dist = px.histogram(only_success, x="marital")
successful_outcome_marital_dist.show()

successful_outcome_marital_dist = px.histogram(only_success, x="education")
successful_outcome_marital_dist.show()


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40195 entries, 0 to 40194
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                40195 non-null  int64  
 1   job                40195 non-null  object 
 2   marital            40195 non-null  object 
 3   education          40195 non-null  object 
 4   no_default_credit  40195 non-null  int64  
 5   housing            40195 non-null  object 
 6   loan               40195 non-null  object 
 7   contact            40195 non-null  object 
 8   month              40195 non-null  object 
 9   day_of_week        40195 non-null  object 
 10  duration           40195 non-null  int64  
 11  campaign           40195 non-null  int64  
 12  pdays              40195 non-null  int64  
 13  previous           40195 non-null  int64  
 14  poutcome           40195 non-null  object 
 15  emp.var.rate       40195 non-null  float64
 16  cons.price.idx     401

In [21]:
# Business Objective 
# Find the model that most accurately predicts if a client will subscribe to the banking service and, if so, at what amount of a term deposit.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [22]:
# Target
X = data.iloc[:, : 7]
y = data["y"]

X.head()

,age,job,marital,education,no_default_credit,housing,loan
0,56,housemaid,married,basic.4y,1,no,no
1,57,services,married,high.school,0,no,no
2,37,services,married,high.school,1,yes,no
3,40,admin.,married,basic.6y,1,no,no
4,56,services,married,high.school,1,no,yes


In [23]:
# Perform encoding and transformations 
# categorical analysis fo features: age, job, marital, education, default, housing, loan

numerical_cols=['age']
categorical_cols=['job', 'marital', 'education', 'no_default_credit', 'housing', 'loan']

numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [25]:
# accuracy of the classifier 
baseline = data['y'].value_counts(normalize = True)[0]
print("baseline: ", baseline)

baseline:  0.8872247792013932


In [26]:
# we should aim to achieve accuracy close to or better than 0.863

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [27]:
# Logistic regression model 

# Create a pipeline with the Processor in the prev step and fit the Logistic Regression model
model = LogisticRegression(max_iter=10000, random_state=42)
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('logreg', model)
])

log_reg_start_time = time.time()
pipe.fit(X_train, y_train)
log_reg_end_time = time.time()

log_reg_train_time = log_reg_end_time - log_reg_start_time
print("Train time for logistic regression: ", log_reg_train_time)

Train time for logistic regression:  2.1797006130218506


### Problem 9: Score the Model

What is the accuracy of your model?

In [28]:
# Make predictions using the model with the test data
preds = pipe.predict(X_test)

In [29]:
# Evaluate the accuracy 
log_reg_test_accuracy = accuracy_score(y_test, preds)
log_reg_train_accuracy = accuracy_score(y_train, pipe.predict(X_train))

print("accuracy score (test): ", log_reg_test_accuracy)
print("accuracy score (train): ", log_reg_train_accuracy )

accuracy score (test):  0.8853138734555104
accuracy score (train):  0.8880437873187376


In [30]:
# We can see that this score is close to our baseline of 0.8634,
# but let's continue to explore other models that may improve our accuracy! 

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [31]:
# Create table with results 
model_comparisons_dict = {'Model':[], # 'Logistic Regression', 'KNN', 'Decision Tree', 'SVM'
        'Train Time':[],
        'Train Accuracy':[],
        'Test Accuracy':[]
       }
model_comparisons = pd.DataFrame(model_comparisons_dict)
# model_comparisons.empty

In [32]:
# KNN with n = 5 default
knn_preprocessor = make_column_transformer((OneHotEncoder(drop = 'if_binary'), categorical_cols), remainder = StandardScaler())

knn_pipeline = Pipeline(steps=[
    ('preprocessor', knn_preprocessor),
    ('knn', KNeighborsClassifier(n_neighbors=5))])

# Fitting the pipeline and measuring time 
knn_start_time = time.time()
knn_pipeline.fit(X_train, y_train)
knn_end_time = time.time()

# Collect metrics 
knn_train_time = knn_end_time - knn_start_time; 

model_comparisons.loc[len(model_comparisons.index)] = ['KNN (n = 5)', knn_train_time, 0, 0] 


In [ ]:
# Score the accuracy 
knn_accuracy_test = knn_pipeline.score(X_test, y_test)
knn_accuracy_train = knn_pipeline.score(X_train, y_train)

print("KNN Train Time: ", dt_train_time)
print("KNN Train Accuracy: ", dt_accuracy_train)
print("KNN Test Accuracy: ", dt_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['KNN (n = 5)', knn_train_time, knn_accuracy_train, knn_accuracy_test] 


In [33]:
# Decision Tree, depth = 3 
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=3))
])
# Fitting the pipeline and measuring time 
dt_start_time = time.time()
dt_pipeline.fit(X_train, y_train)
dt_end_time = time.time()

# Collect metrics 
dt_train_time = dt_end_time - dt_start_time; 
dt_accuracy_test = dt_pipeline.score(X_test, y_test)
dt_accuracy_train = dt_pipeline.score(X_train, y_train)

print("Decision Tree Train Time: ", dt_train_time)
print("Decision Tree Train Accuracy: ", dt_accuracy_train)
print("Decision Tree Test Accuracy: ", dt_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['Decision Tree (depth = 3)', dt_train_time, dt_accuracy_train,dt_accuracy_test] 

Decision Tree Train Time:  0.1601884365081787
Decision Tree Train Accuracy:  0.888399203866932
Decision Tree Test Accuracy:  0.8853138734555104


In [34]:
# SVM model - linear
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear'))
])

# Fitting the pipeline  and measuring time 
svm_start_time = time.time()
svm_pipeline.fit(X_train, y_train)
svm_end_time = time.time()

# Collect metrics 
svm_train_time = svm_end_time - svm_start_time; 
svm_accuracy_test = svm_pipeline.score(X_test, y_test)
svm_accuracy_train = svm_pipeline.score(X_train, y_train)

print("SVM Train Time: ", svm_train_time)
print("SVM Train Accuracy: ", svm_accuracy_train)
print("SVM Test Accuracy: ", svm_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['SVM (linear)', svm_train_time, svm_accuracy_train, svm_accuracy_test] 

SVM Train Time:  37.847251653671265
SVM Train Accuracy:  0.8880437873187376
SVM Test Accuracy:  0.8853138734555104


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [35]:
experiment_model_comparisons = model_comparisons
experiment_model_comparisons.head()

,Model,Train Time,Train Accuracy,Test Accuracy
0,KNN (n = 5),0.114878,0.000000,0.000000
1,Decision Tree (depth = 3),0.160188,0.888399,0.885314
2,SVM (linear),37.847252,0.888044,0.885314


In [36]:
# KNN Iteration: increase neighbor size 
n_values = [5, 10, 20, 50]

for n in n_values: 
    knn_preprocessor = make_column_transformer((OneHotEncoder(drop = 'if_binary'), categorical_cols), remainder = StandardScaler())

    knn_pipeline = Pipeline(steps=[
        ('preprocessor', knn_preprocessor),
        ('knn', KNeighborsClassifier(n_neighbors=n))])

    # Fitting the pipeline and measuring time 
    knn_start_time = time.time()
    knn_pipeline.fit(X_train, y_train)
    knn_end_time = time.time()
    
#     knn_accuracy_test = knn_pipeline.score(X_test, y_test)
#     knn_accuracy_train = knn_pipeline.score(X_train, y_train)
    knn_train_time = knn_end_time - knn_start_time; 

    print("KNN Train Time: ", knn_train_time)
#     print("KNN Train Accuracy: ", knn_accuracy_train)
#     print("KNN Test Accuracy: ", knn_accuracy_test)

    # Collect metrics
    model_name = "KNN (n = " + str(n) + ")"
    experiment_model_comparisons.loc[len(experiment_model_comparisons.index)] = [model_name, knn_train_time, 0, 0] 


KNN Train Time:  0.1203453540802002
KNN Train Time:  0.11224937438964844
KNN Train Time:  0.11286735534667969
KNN Train Time:  0.11077070236206055


In [37]:
# Decision Tree Iteration: increase depth size
depths = [5, 10, 50, 100]

for depth in depths:
    dt_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(max_depth=depth))
    ])
    # Fitting the pipeline and measuring time 
    dt_start_time = time.time()
    dt_pipeline.fit(X_train, y_train)
    dt_end_time = time.time()

    # Collect metrics 
    dt_train_time = dt_end_time - dt_start_time; 
    dt_accuracy_test = dt_pipeline.score(X_test, y_test)
    dt_accuracy_train = dt_pipeline.score(X_train, y_train)

    print("Decision Tree Train Time: ", dt_train_time)
    print("Decision Tree Train Accuracy: ", dt_accuracy_train)
    print("Decision Tree Test Accuracy: ", dt_accuracy_test)
    
    model_name = "Decision Tree (depth = " + str(depth) + ")"

    # Add results to table
    experiment_model_comparisons.loc[len(experiment_model_comparisons.index)] = [model_name, dt_train_time, dt_accuracy_train,dt_accuracy_test] 


Decision Tree Train Time:  0.1797952651977539
Decision Tree Train Accuracy:  0.8890744953085016
Decision Tree Test Accuracy:  0.8856455759184012
Decision Tree Train Time:  0.2663230895996094
Decision Tree Train Accuracy:  0.8939437020187659
Decision Tree Test Accuracy:  0.883240733062443
Decision Tree Train Time:  0.7539639472961426
Decision Tree Train Accuracy:  0.9181475689508104
Decision Tree Test Accuracy:  0.8634215109047184
Decision Tree Train Time:  0.7547657489776611
Decision Tree Train Accuracy:  0.9181475689508104
Decision Tree Test Accuracy:  0.8636702877518866


In [38]:
# SVM Iteration: change kernel type (rbf and poly)
kernels = ['rbf', 'poly']

for kernel in kernels:
    # SVM model - linear
    svm_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', SVC(kernel=kernel))
    ])

    # Fitting the pipeline  and measuring time 
    svm_start_time = time.time()
    svm_pipeline.fit(X_train, y_train)
    svm_end_time = time.time()

    # Collect metrics 
    svm_train_time = svm_end_time - svm_start_time; 
    svm_accuracy_test = svm_pipeline.score(X_test, y_test)
    svm_accuracy_train = svm_pipeline.score(X_train, y_train)

    print("SVM Train Time: ", svm_train_time)
    print("SVM Train Accuracy: ", svm_accuracy_train)
    print("SVM Test Accuracy: ", svm_accuracy_test)
    
    model_name = "SVM (" + kernel + ")"

    # Add results to table
    experiment_model_comparisons.loc[len(experiment_model_comparisons.index)] = [model_name, svm_train_time, svm_accuracy_train, svm_accuracy_test] 
    

SVM Train Time:  17.089672803878784
SVM Train Accuracy:  0.8880437873187376
SVM Test Accuracy:  0.8853138734555104
SVM Train Time:  36.379918336868286
SVM Train Accuracy:  0.8880437873187376
SVM Test Accuracy:  0.8853138734555104


In [39]:
# View all results 
print(experiment_model_comparisons)
print("baseline: ", baseline)
print("Log Reg Train Accuracy: ", log_reg_train_accuracy)
print("Log Reg Test Accuracy: ", log_reg_test_accuracy)

                          Model  Train Time  Train Accuracy  Test Accuracy
0                   KNN (n = 5)    0.114878        0.000000       0.000000
1     Decision Tree (depth = 3)    0.160188        0.888399       0.885314
2                  SVM (linear)   37.847252        0.888044       0.885314
3                   KNN (n = 5)    0.120345        0.000000       0.000000
4                  KNN (n = 10)    0.112249        0.000000       0.000000
5                  KNN (n = 20)    0.112867        0.000000       0.000000
6                  KNN (n = 50)    0.110771        0.000000       0.000000
7     Decision Tree (depth = 5)    0.179795        0.889074       0.885646
8    Decision Tree (depth = 10)    0.266323        0.893944       0.883241
9    Decision Tree (depth = 50)    0.753964        0.918148       0.863422
10  Decision Tree (depth = 100)    0.754766        0.918148       0.863670
11                    SVM (rbf)   17.089673        0.888044       0.885314
12                   SVM 

# Conclusions 
We will now look at the whole analysis to determine which model could most accurately predict the outcome of a marketing call. 


## Data Outcomes 

We can see from some preliminary analysis that admins, marries, and with university degrees. This provides interesting context for the types of users taking bank credit loans and subscribing to this service. This helped as a point of comparison when looking at the predictions from the various models. 


## Best Model 
Compared to the baseline model that was generated, the Logistic Regression model still seems to have the highest accuracy. The Logistic Regression train accuracy was 0.8880 and test accuracy was 0.8853, which both compare very well to the baseline of 0.8872. 

SVM models seem to take the most time regardless of if the model is a linear, rbf, or poly model. 

The Decision Trees have higher accuracies but that is a result of higher tree depths which is an example of overfitting. 